In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import ElasticVectorSearch
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.document_loaders import TextLoader
import os

In [ ]:
HF_SENTENCE_TRANSFORMER_MODEL = "sentence-transformers/all-mpnet-base-v2"
ELASTIC_URL = "http://localhost:9200/"
MONGO_URL = "http://localhost:27017/"
ELASTIC_INDEX = "genshinpedia_small"
DB_FILES = "en_datasets"
CACHE_DIR = "./cache"
T2T_MODEL = "google/flan-t5-base"

In [ ]:
def load_sentences_hf_model():
    print("Preparing Huggingface embedding setup...")
    return HuggingFaceEmbeddings(model_name=HF_SENTENCE_TRANSFORMER_MODEL)

In [ ]:
# Creacion del modelo para generacion de texto
def get_t5_model():
    print("Preparing model {}".format(T2T_MODEL))
    tokenizer = AutoTokenizer.from_pretrained(T2T_MODEL)
    model = AutoModelForSeq2SeqLM.from_pretrained(T2T_MODEL, cache_dir=CACHE_DIR) 
    pipe = pipeline(
        "text2text-generation",
        model=model, 
        tokenizer=tokenizer, 
        max_length=100
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

In [ ]:
elastic_db = ElasticVectorSearch(embedding=load_sentences_hf_model(), 
                                 elasticsearch_url=ELASTIC_URL,
                                 index_name=ELASTIC_INDEX)

In [ ]:
ORIGINAL_PROMPT = """
I am a helpful AI that answers questions. When I don't know the answer I say I don't know. 
I know context: {context} when asked: {question} my response using only information in the context is: 
"""
TUNED_PROMPT = """
I am a helpful AI that answers questions providing details from the context in my answer. When I don't know the answer I say "I don't know" without the quotes.
I know the context: {context} when user ask me the {question}. My detailed response is:
"""

MEMORY_PROMPT = """
I am a helpful AI that answers questions providing details from the context in my answer. When I don't know the answer I say "I don't know" without the quotes.
I know the context {chat_history} {context} when user asks me {question}, my detailed response is:
"""

In [ ]:
def make_the_llm():
    prompt_informed = PromptTemplate(template=TUNED_PROMPT, 
                                     input_variables=["context", "question"])
    llm = get_t5_model()
    return LLMChain(llm=llm, prompt=prompt_informed, verbose=True)

In [ ]:
def make_memory_llm(memory):
    prompt_informed = PromptTemplate(template=MEMORY_PROMPT, 
                                     input_variables=["context", "chat_history", "question"])
    llm = get_t5_model()
    return LLMChain(llm=llm, prompt=prompt_informed, verbose=True, memory=memory)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history",
                                  input_key="context")

In [ ]:
llm_chain_informed = make_the_llm()
#llm_with_memory = make_memory_llm()
#llm_with_memory = make_memory_llm(memory)

In [ ]:
def ask_a_question(question):
    similar_docs = elastic_db.similarity_search(question)
    question_context = similar_docs[0].page_content + "\n" + similar_docs[1].page_content
    #print("Best constructed context: {}".format(question_context))
    ## Ask Local LLM context informed prompt
    informed_context = question_context #similar_docs[0].page_content
    #informed_response = llm_chain_informed.run(context=informed_context,question=question)
    #informed_response = agent_chain.run(context=informed_context, question=question)
    informed_response = llm_with_memory.run(context = informed_context, question = question)
    
    return informed_response

In [ ]:
test_questions = ["Who is the Dendro Archon?"]#, "Where does Kirara works?", "Where is Kaeya from?" ]

In [ ]:
for q in test_questions:
    print("My Answer is: {}".format(ask_a_question(q)))

In [ ]:
ask_a_question("Where does Kirara works?")

In [ ]:
ask_a_question("What does she wears around her waist?")

In [ ]:
ask_a_question("Whats her alternative feline form?")